In [5]:
def io_file():
    import os
    
    path = os.getcwd()
    # 추출된 data 저장할 디렉토리
    mkdir_path = path +"/output/"

    # 디렉토리 생성
    if not os.path.exists(mkdir_path):
        os.mkdir(mkdir_path)
    
    # 읽어올 file 경로에서 모든 파일 읽어옴
    files = os.listdir(path+"/file")
    # 확장자가 csv 인것만 저장
    files_csv = [file for file in files if file[-3:] == 'csv']
    del(files)
    
    return files_csv
    
    
def tran_ip(cidrs):
    import ipaddress
    #임시 변수 
    result_list=[]
    
    # CIDR(subnet) 기반으로 IP 리스트로 변경
    for cidr in cidrs:
        result_list.extend([str(ip) for ip in ipaddress.ip_network(cidr)])
    
    # Return value 에 IP 리스트 추가
    cidrs.extend(result_list)
    # 임시 변수 삭제
    del(result_list)
    
    return cidrs

def extract_data(cidrs, files):
    import pandas as pd
    df = pd.DataFrame()

    for file in files:
        data = pd.read_csv('./file/'+file)
        #하나의 데이터 프레임으로 합치기 위한 코드
        df = pd.concat([df, data])
    
    # pandas query 실행 시 column 명에 공백이 있어 오류 발생으로 column 명 재정의
    df.rename(columns = {'Src Address':'Src_Address','Dst Address':'Dst_Address'},inplace=True)
    # List 변수안에 있는 내용이 column이 있을 경우 Row 추가
    df = df.query('Src_Address in @ip_range or Dst_Address in @ip_range')
    
    return df

def extract_data_one_select(cidrs, files):
    import pandas as pd
    import os 
    df = pd.DataFrame()
    
    # 데이터 추출 완료 후 저장할 파일 위치 
    mkdir_path = os.getcwd() + "/output/"

    for file in files:
        #특정 위치 밑 파일 하나씩 읽어옴
        data = pd.read_csv('./file/'+file)
        
        # CSV 로 저장하기 위한 임시 변수 선언
        file_name = mkdir_path + file
        # pandas query 실행 시 column 명에 공백이 있어 오류 발생으로 column 명 재정의
        data.rename(columns = {'Src Address':'Src_Address','Dst Address':'Dst_Address'},inplace=True)
        # List 변수안에 있는 내용이 column이 있을 경우 Row 추가
        data = data.query('Src_Address in @ip_range or Dst_Address in @ip_range')
        
        # CSV로 추출 결과 저장
        data.to_csv(file_name,encoding='utf-8')

if __name__ == '__main__':
    import pandas as pd
    cidrs = ['10.248.193.0/24', '10.168.178.0/24', '10.84.84.0/24', '10.168.90.0/24', '10.168.88.0/24']

    files = io_file()
    ip_range = tran_ip(cidrs)
    data = extract_data(ip_range, files)
    extract_data_one_select(ip_range, files)


In [6]:
def extract_data(cidrs, files):
    import pandas as pd
    df = pd.DataFrame()

    for file in files:
        data = pd.read_csv('./file/'+file)
        #하나의 데이터 프레임으로 합치기 위한 코드
        df = pd.concat([df, data])
    
    # pandas query 실행 시 column 명에 공백이 있어 오류 발생으로 column 명 재정의
    print('Total Rows = {data}'.format(data=df.shape[0]))
    df.rename(columns = {'Src Address':'Src_Address','Dst Address':'Dst_Address'},inplace=True)
    # List 변수안에 있는 내용이 column이 있을 경우 Row 추가
    df = df.query('Src_Address in @ip_range or Dst_Address in @ip_range')
    
    return df

In [7]:
import pandas as pd
cidrs = ['10.248.193.0/24', '10.168.178.0/24', '10.84.84.0/24', '10.168.90.0/24', '10.168.88.0/24']

files = io_file()
ip_range = tran_ip(cidrs)
data = extract_data(ip_range, files)
print('Extract data Rows = {data}'.format(data=data.shape[0]))
# extract_data_one_select(ip_range, files)

Total Rows = 998
Extract data Rows = 53


In [8]:
data.head(100)

,Unnamed: 0,Src Zone,Src_Address,Dst Zone,Dst_Address,Service,App,URL,Action,Last Matched (days ago),사용유무,사용우무
8,8,trust,10.168.90.88,untrust,10.168.7.28,tcp_1433,NaN,NaN,permit,419.0,NaN,NaN
9,9,NaN,10.168.90.89,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21,21,NaN,10.168.88.0/24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23,23,NaN,10.168.90.0/24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26,26,NaN,10.168.88.0/24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28,28,NaN,10.168.90.0/24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
31,31,trust,10.168.88.0/24,untrust,192.168.252.21,tcp_139,NaN,NaN,permit,266.0,NaN,NaN
33,33,NaN,10.168.90.0/24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
35,35,trust,10.168.90.0/24,untrust,192.168.48.241,tcp_8888,NaN,NaN,permit,NaN,NaN,NaN
37,37,trust,10.168.88.0/24,untrust,192.168.48.241,tcp_8888,NaN,NaN,permit,NaN,NaN,NaN
